In [2]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.argv=['']
del sys
os.chdir("../")

In [35]:
import matplotlib.pyplot as plt
import os
import json
import math
import torch
import logging
import numpy as np
import pandas as pd

from IPython import embed
from common import data_preprocess 
from common.utils import print_to_json
from common.dataloader import load_dataset
from common.sliding import BatchSlidingWindow, WindowIterator
from common.config import parse_arguments, set_logger, initialize_config
from networks.mlstm_token_clf import MultiLSTMEncoder
from torch import nn
from glob import glob
from datetime import datetime

%matplotlib inline

In [90]:
wadi_path = "./datasets/anomaly/WADI"

In [91]:
train = pd.read_csv(os.path.join(wadi_path, "train.csv"), skiprows=4)
test = pd.read_csv(os.path.join(wadi_path, "test.csv"))
labels = pd.read_csv(os.path.join(wadi_path, "labels_raw.csv"), sep=" ")
print(train.shape, test.shape)

(1209601, 130) (172801, 130)


In [92]:
def merge_date_label(t1, t2):
    date_time_str = t1 + " "+ t2
    return datetime.strptime(date_time_str, '%d/%m/%y %H:%M:%S')
def merge_date_data(t1, t2):
    date_time_str = t1 + " "+ t2
    return datetime.strptime(date_time_str, '%d/%m/%Y %H:%M:%S')

labels["StartTime"] = labels.apply(lambda row: merge_date(row["StartingTime1"], row["StartingTime2"]), axis=1)
labels["EndTime"] = labels.apply(lambda row: merge_date(row["EndingTime1"], row["EndingTime2"]), axis=1)
test["DateTime"] = test.apply(lambda row: merge_date_data(row["Date"], row["Time"][0:7]), axis=1)

In [93]:
testv = test.values
anomaly_intervals = list(zip(labels["StartTime"],labels["EndTime"]  ))
label_list = []
for i in range(test.shape[0]):
    if any([s<= testv[i, -1] <= e for s,e in anomaly_intervals] ):
        label_list.append(1)
    else:
        label_list.append(0)
print("anomaly rate: {:.3f}%".format(100*np.array(label_list).sum() / len(label_list)))

anomaly rate: 3.634%


In [99]:
trainf = train.drop(["Row", "Date", "Time"], axis=1)
testf = test.drop(["Row", "Date", "Time", "DateTime"], axis=1)

In [104]:
trainf.describe()

,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_002_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_003_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_004_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_AIT_005_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_FIT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_LS_001_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_LS_002_AL,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_LT_001_PV,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\1_MV_001_STATUS,...,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_MV_001_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_MV_002_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_MV_003_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_001_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_002_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_003_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\3_P_004_STATUS,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\LEAK_DIFF_PRESSURE,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\PLANT_START_STOP_LOG,\\WIN-25J4RO10SBF\LOG_DATA\SUTD_WADI\LOG_DATA\TOTAL_CONS_REQUIRED_FLOW
count,1.209601e+06,1.209589e+06,1.209601e+06,1.209595e+06,1.209601e+06,1.209601e+06,1209601.0,1209601.0,1.209601e+06,1.209601e+06,...,1209601.0,1209601.0,1209601.0,1209601.0,1209601.0,1209601.0,1209601.0,1.209601e+06,1209601.0,1.209601e+06
mean,1.676346e+02,6.215798e-01,1.175245e+01,4.877939e+02,3.011777e-01,5.246331e-01,0.0,0.0,5.691863e+01,1.270222e+00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,6.302362e+01,1.0,5.486840e-01
std,1.310815e+01,5.920273e-02,1.916769e-01,2.334323e+01,4.851459e-02,8.523658e-01,0.0,0.0,1.157742e+01,4.484743e-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.869981e+00,0.0,4.482662e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.083020e-01,7.896870e-04,0.0,0.0,2.653000e-02,0.000000e+00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.539650e+01,1.0,0.000000e+00
25%,1.564740e+02,5.834770e-01,1.164640e+01,4.752790e+02,2.602440e-01,1.107680e-03,0.0,0.0,4.883670e+01,1.000000e+00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,6.048440e+01,1.0,2.200000e-01
50%,1.659010e+02,6.254710e-01,1.178810e+01,4.929870e+02,2.992290e-01,1.188810e-03,0.0,0.0,5.729140e+01,1.000000e+00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,6.171660e+01,1.0,5.300000e-01
75%,1.746570e+02,6.614670e-01,1.183760e+01,5.039210e+02,3.381910e-01,1.854170e+00,0.0,0.0,6.292250e+01,2.000000e+00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,6.365020e+01,1.0,7.200000e-01
max,2.143110e+02,2.059320e+00,1.218070e+01,5.265290e+02,4.217350e-01,2.077280e+00,0.0,0.0,1.002170e+02,2.000000e+00,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.472950e+02,1.0,2.260000e+00


In [ ]:
trainf.describe()